<a href="https://colab.research.google.com/github/anandxrfeu/anandxrfeu.github.io/blob/main/project_1/lm_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Project 1: Build an LLM Playground

Welcome! In this project, you’ll learn foundations of large language models (LLMs). We’ll keep the code minimal and the explanations high‑level so that anyone who can run a Python cell can follow along.  

We'll be using Google Colab for this project. Colab is a free, browser-based platform that lets you run Python code and machine learning models without installing anything on your local computer. Click the button below to open this notebook directly in Google Colab and get started!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bytebyteai/ai-eng-projects/blob/main/project_1/lm_playground.ipynb)

---
## Learning Objectives  
* **Tokenization** and how raw text is tokenized into a sequene of discrete tokens
* Inspect **GPT2** and **Transformer architecture**
* Loading pre-trained LLMs using **Hugging Face**
* **Decoding strategies** to generate text from LLMs
* Completion versus **intrusction fine-tuned** LLMs


Let's get started!

In [1]:
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)

torch 2.8.0+cu126 | transformers 4.56.2


# 1 - Tokenization

A neural network can’t digest raw text. They need **numbers**. Tokenization is the process of converting text into IDs. In this section, you'll learn how tokenization is implemented in practice.

Tokenization methods generally fall into three categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word‑level tokenization

Split text on whitespace and store each **word** as a token.

In [22]:
# 1. Tiny corpus
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# 2. Build the vocabulary
PAD, UNK = "[PAD]", "[UNK]"
vocab = []
word2id = {}
id2word = {}

vocab.append(PAD)
vocab.append(UNK)

for sentence in corpus:
  vocab.extend(sentence.split())


print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])

for index, word in enumerate(vocab):
    id2word[index] = word


word2id = {v: k for k, v in id2word.items()}


# 3. Encode / decode
def encode(text):
  token = []
  for word in text.split():
    id = word2id.get(word, 1)
    token.append(id)

  return token


def decode(ids):
  response = []
  for id in ids:
    response.append(id2word[id])
  return " ".join(response)


# 4. Demo
sample = "The brown unicorn jumps"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)



Vocabulary size: 23 words
First 15 vocab entries: ['[PAD]', '[UNK]', 'The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', 'Tokenization', 'converts', 'text', 'to']

Input text : The brown unicorn jumps
Token IDs  : [2, 4, 1, 6]
Decoded    : The brown [UNK] jumps


Word-level tokenization has two major limitations:
1. Large vocabulary size
2. Out-of-vocabulary (OOV) issue

### 1.2 - Character‑level tokenization

Every single character (including spaces and emojis) gets its own ID. This guarantees zero out‑of‑vocabulary issues but very long sequences.

In [66]:
# 1.2 — Character-level tokenization

import string

# 1. Build a fixed vocabulary: a–z + A–Z + specials
vocab = list(string.ascii_lowercase + string.ascii_uppercase)
vocab.extend(["<PAD>", "<UNK>"])

# Create lookup dictionaries
char2id = {ch: i for i, ch in enumerate(vocab)}
id2char = {i: ch for ch, i in char2id.items()}

print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")

# 2. Encode / decode
def encode(text):
    return [char2id.get(ch, char2id["<UNK>"]) for ch in text]

def decode(ids):
    return "".join(id2char.get(i, "<UNK>") for i in ids)

# 3. Demo
sample = "Hello"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


Vocabulary size: 54 (52 letters + 2 specials)

Input text : Hello
Token IDs  : [33, 4, 11, 11, 14]
Decoded    : Hello


### 1.3 - Subword‑level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** the most common character and gorup them into new tokens. For example, the word `unbelievable` might turn into three tokens: `["un", "believ", "able"]`. This approach strikes a balance between word-level and character-level methods and fix their limitations.

For example, `BPE` algorithm forms the vocabulary using the following steps:
1. **Start with bytes** → every character is its own token.  
2. **Count all adjacent pairs** in a huge corpus.  
3. **Merge the most frequent pair** into a new token.  
   *Repeat steps 2-3* until you hit the target vocab size (e.g., 50 k).

Let's see `BPE` in practice.

In [38]:
# 1. Load a pretrained BPE tokenizer (GPT-2 uses BPE).
# Refer to  https://huggingface.co/docs/transformers/en/fast_tokenizers

from transformers import AutoTokenizer

bpe_tok = AutoTokenizer.from_pretrained("gpt2")


# 2. Encode / decode
def encode(text):
    return bpe_tok(text)  # returns lists

def decode(ids):
    return bpe_tok.decode(ids["input_ids"])

# 3. Demo
sample = "Unbelievable tokenization powers! 🚀"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Tokens     :", bpe_tok.convert_ids_to_tokens(ids["input_ids"]))
print("Count     :", len(bpe_tok.convert_ids_to_tokens(ids["input_ids"])))
print("Decoded    :", recovered)





Input text : Unbelievable tokenization powers! 🚀
Token IDs  : {'input_ids': [3118, 6667, 11203, 540, 11241, 1634, 5635, 0, 12520, 248, 222], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Tokens     : ['Un', 'bel', 'iev', 'able', 'Ġtoken', 'ization', 'Ġpowers', '!', 'ĠðŁ', 'ļ', 'Ģ']
Count     : 11
Decoded    : Unbelievable tokenization powers! 🚀


### 1.4 - TikToken

`tiktoken` is a production-ready library which offers high‑speed tokenization used by OpenAI models.  
Let's compare the older **gpt2** encoding with the newer **cl100k_base** used in GPT‑4.

In [37]:
# Use gpt2 and cl100k_base to encode and decode the following text
# Refer to https://github.com/openai/tiktoken
import tiktoken


#sentence = "The 🌟 star-player scored 40 points!"
sentence = "https://example.com/path?x=1&y=2"

# 1. Load GPT-2 BPE and cl100k_base tokenizers
gpt_tok = tiktoken.get_encoding("gpt2")
cbase_tok = tiktoken.get_encoding("cl100k_base")

# 2. Encode / Decode
def encode(tok, text):
  return tok.encode(text)

def decode(tok, token_ids):
  return tok.decode(token_ids)

# 3. Compare
for name, tok in [("GPT-2 (BPE)", gpt_tok), ("cl100k_base (GPT-4)", cbase_tok)]:
    ids = encode(tok, sentence)
    decoded = decode(tok, ids)
    print(f"\n{name}")
    print("IDs      :", ids)
    print("Count    :", len(ids))
    print("Decoded  :", decoded)




GPT-2 (BPE)
IDs      : [5450, 1378, 20688, 13, 785, 14, 6978, 30, 87, 28, 16, 5, 88, 28, 17]
Count    : 15
Decoded  : https://example.com/path?x=1&y=2

cl100k_base (GPT-4)
IDs      : [2485, 1129, 8858, 916, 52076, 30, 87, 28, 16, 5, 88, 28, 17]
Count    : 13
Decoded  : https://example.com/path?x=1&y=2


Experiment: try new sentences, emojis, code snippets, or other languages. If you are interested, try implementing the BPE algorithm yourself.

### 1.5 - Key Takeaways

* **Word‑level**: simple but brittle (OOV problems).  
* **Character‑level**: robust but produces long sequences.  
* **BPE / Byte‑Level BPE**: middle ground used by most LLMs.  
* **tiktoken**: shows how production models tokenize with pre‑trained sub‑word vocabularies.

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer applies a simple operation (matrix multiplication, attention, etc.). Stacking hundreds of these layers lets the model capture patterns and statistical relations from text. The final output is a vector of scores that says, “how likely is each possible token to come next?”

> Think of the whole network as **one gigantic equation** whose parameters were tuned during training to minimize prediction error.



### 2.1 - A Single `Linear` Layer

Before we explore Transformer, let’s start tiny:

* A **Linear layer** performs `y = Wx + b`  
  * `x` – input vector  
  * `W` – weight matrix (learned)  
  * `b` – bias vector (learned)

Although this looks basic, chaining thousands of such linear transforms (with nonlinearities in between) gives neural nets their expressive power.


In [41]:
import torch.nn as nn

class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        # define learnable parameters
        self.W = nn.Parameter(torch.randn(out_features, in_features))
        self.b = nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        # y = Wx + b
        return x @ self.W.T + self.b

In [42]:
import torch.nn as nn, torch

lin = nn.Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[-0.3548, -0.1896,  0.4094],
        [-0.0558,  0.2814,  0.0372]], requires_grad=True)
Bias   : Parameter containing:
tensor([ 0.2582, -0.3858], requires_grad=True)
Output : tensor([ 0.2976, -0.7044], grad_fn=<ViewBackward0>)


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

Below, we load the smallest public GPT-2 (124 M parameters), grab its *first* block, and inspect the pieces.


In [45]:
import torch
from transformers import GPT2LMHeadModel

# Load the 124 M-parameter GPT-2 and inspect its layers (12 layers)
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
n_layers = len(gpt2.transformer.h)
first_block = gpt2.transformer.h[0]        # Transformer block 0
print("Layers:", n_layers)
print("Block parts:", type(first_block.attn).__name__, "|", type(first_block.mlp).__name__)


Layers: 12
Block parts: GPT2Attention | GPT2MLP


In [46]:
# Run a tiny forward pass through the first block
seq_len = 8
vocab_size = gpt2.config.vocab_size
hidden_size = gpt2.config.n_embd

dummy_tokens = torch.randint(0, vocab_size, (1, seq_len))

with torch.no_grad():
    # Token + positional embeddings (GPT-2 style)
    tok_emb = gpt2.transformer.wte(dummy_tokens)                                  # (1, L, D)
    pos_ids = torch.arange(seq_len, dtype=torch.long).unsqueeze(0)                # (1, L)
    pos_emb = gpt2.transformer.wpe(pos_ids)                                       # (1, L, D)
    hidden = tok_emb + pos_emb                                                    # (1, L, D)

    # Single Transformer block forward (causal mask handled internally)
    out = first_block(hidden)                                                     # returns hidden_states or tuple
    out = out[0] if isinstance(out, tuple) else out

print("\nOutput shape :", out.shape) # (batch, seq_len, hidden_size)


Output shape : torch.Size([1, 8, 768])


### 2.3 - Inside GPT-2

GPT-2 is just many of those modules arranged in a repeating *block*. Let's print the modules inside the Transformer.

In [48]:
# Print the name and modules inside gpt2
"""
YOUR CODE HERE
"""
for name, module in gpt2.named_modules():
    print(f"{name:40s} {type(module).__name__}")

                                         GPT2LMHeadModel
transformer                              GPT2Model
transformer.wte                          Embedding
transformer.wpe                          Embedding
transformer.drop                         Dropout
transformer.h                            ModuleList
transformer.h.0                          GPT2Block
transformer.h.0.ln_1                     LayerNorm
transformer.h.0.attn                     GPT2Attention
transformer.h.0.attn.c_attn              Conv1D
transformer.h.0.attn.c_proj              Conv1D
transformer.h.0.attn.attn_dropout        Dropout
transformer.h.0.attn.resid_dropout       Dropout
transformer.h.0.ln_2                     LayerNorm
transformer.h.0.mlp                      GPT2MLP
transformer.h.0.mlp.c_fc                 Conv1D
transformer.h.0.mlp.c_proj               Conv1D
transformer.h.0.mlp.act                  NewGELUActivation
transformer.h.0.mlp.dropout              Dropout
transformer.h.1                   

As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

Passing a token sequence through an **LLM** yields a tensor of **logits** with shape  
`(batch_size, seq_len, vocab_size)`.  
Applying `softmax` on the last dimension turns those logits into probabilities.

The cell below feeds an 8-token dummy sequence, prints the logits shape, and shows the five most likely next tokens for the final position.


In [49]:
# 2.4 — LLM's output
import torch, torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load gpt2 model and tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

# Tokenize input text
text = "Hello my name"
inputs = tokenizer(text, return_tensors="pt")

# Get logits by passing the ids to the gpt2 model
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits  # (batch_size, seq_len, vocab_size)

print("Logits shape :", logits.shape)

# Predict next token (use last position)
next_token_logits = logits[0, -1]                           # (vocab_size,)
probs = F.softmax(next_token_logits, dim=-1)                # probabilities
top_probs, top_ids = torch.topk(probs, k=5)                 # top-5

print("\nTop-5 predictions for the next token:")
for p, i in zip(top_probs.tolist(), top_ids.tolist()):
    token_str = tokenizer.decode([i]).strip()
    print(f"{i:>6}  p={p:.4f}  {repr(token_str)}")


Logits shape : torch.Size([1, 3, 50257])

Top-5 predictions for the next token:
   318  p=0.7773  'is'
    11  p=0.0373  ','
   338  p=0.0332  "'s"
   373  p=0.0127  'was'
   290  p=0.0076  'and'


### 2.5 - Key Takeaway

A language model is nothing mystical: it’s a *huge composition* of small, understandable layers trained to predict the next token in a sequence of tokens.

# 3 - Generation
Once an LLM is trained to predict the probabilities, we can generate text from the model. This process is called decoding or sampling.

At each step, the LLM outputs a **probability distribution** over the next token. It is the job of the decoding algorithm to pick the next token, and move on to the next token. There are different decoding algorithms and hyper-parameters to control the generaiton:
* **Greedy** → pick the single highest‑probability token each step (safe but repetitive).  
* **Top‑k / Nucleus (top‑p)** → sample from a subset of likely tokens (adds variety).
* **beam** -> applies beam search to pick tokens
* **Temperature** → a *creativity* knob. Higher values flatten the probability distribution.

### 3.1 - Greedy decoding

In [59]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
}
tokenizers, models = {}, {}
# Load models and tokenizers
for key, name in MODELS.items():
    tok = AutoTokenizer.from_pretrained(name)
    mdl = AutoModelForCausalLM.from_pretrained(name)
    if tok.pad_token_id is None:
        tok.pad_token = tok.eos_token
        mdl.config.pad_token_id = tok.eos_token_id
    mdl.eval()
    mdl.to("cuda" if torch.cuda.is_available() else "cpu")
    tokenizers[key] = tok
    models[key] = mdl

def generate(model_key, prompt, strategy="greedy", max_new_tokens=100,
             temperature=1.0, top_k=50, top_p=0.9, num_beams=5):
    tok, mdl = tokenizers[model_key], models[model_key]
    # Return the generations based on the provided strategy: greedy, top_k, top_p
    device = next(mdl.parameters()).device
    inputs = tok(prompt, return_tensors="pt").to(device)

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id,
    )

    if strategy == "greedy":
        gen_kwargs.update(do_sample=False)
    elif strategy == "top_k":
        gen_kwargs.update(do_sample=True, top_k=top_k, temperature=temperature)
    elif strategy == "top_p":
        gen_kwargs.update(do_sample=True, top_p=top_p, temperature=temperature)
    elif strategy == "beam":
        gen_kwargs.update(do_sample=False, num_beams=num_beams, early_stopping=True)
    else:
        raise ValueError("strategy must be one of: greedy | top_k | top_p | beam")

    with torch.no_grad():
        out_ids = mdl.generate(**inputs, **gen_kwargs)

    return tok.decode(out_ids[0], skip_special_tokens=True)


In [60]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Greedy ==")
    print(generate("gpt2", prompt, "greedy", 80))



== GPT-2 | Greedy ==
Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== GPT-2 | Greedy ==
What is 2+2?

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

== GPT-2 | Greedy ==
Suggest a party theme.

The party theme is a simple, simple, and fun way to get your friends to join you.

The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple


Naively picking the single best token every time has the following issues in practice:

* **Loop**: “The cat is is is…”  
* **Miss long-term payoff**: the highest-probability word *now* might paint you into a boring corner later.

### 3.2 - Top-k or top-p sampling

In [61]:

tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Top-p ==")
    print(generate("gpt2", prompt, "top_p", 40))



== GPT-2 | Top-p ==
Once upon a time, it seemed the only way to bring back any of the fallen heroes were to save the life of one of the dead. But when he was done, something happened, something the world of Heroes would

== GPT-2 | Top-p ==
What is 2+2?

2+2 is the basic question. When you think about a person's level of intelligence you usually want to see a person of similar intelligence. But when you consider their abilities, such as

== GPT-2 | Top-p ==
Suggest a party theme.

A party theme is the character theme for the party, although the following rules are included:

When you are in a party, leave your companions behind when you leave for a new party


### 3.3 - Try It Yourself

1. Scroll to the list called `tests`.
2. Swap in your own prompts or tweak the decoding strategy.  
3. Re‑run the cell and compare the vibes.

> **Tip:** Try the same prompt with `greedy` vs. `top_p` (0.9) and see how the tone changes. Notice especially how small temperature tweaks can soften or sharpen the prose.

* `strategy`: `"greedy"`, `"beam"`, `"top_k"`, `"top_p"`  
* `temperature`: `0.2 – 2.0`  
* `k` or `p` thresholds



# 4 - Completion vs. Instruction-tuned LLMs

We have seen that we can use GPT2 model to pass an input text and generate a new text. However, this model only continues the provided text. It is not engaging in a dialouge-like conversation and cannot be helpful by answering instructions. On the other hand, **instruction-tuned LLMs** like `Qwen-Chat` go through an extra training stage called **post-training** after the base “completion” model is finished. Because of post-training step, an instruction-tuned LLM will:

* **Read the entire prompt as a request,** not just as text to mimic.  
* **Stay in dialogue mode**. Answer questions, follow steps, ask clarifying queries.  
* **Refuse or safe-complete** when instructions are unsafe or disallowed.  
* **Adopt a consistent persona** (e.g., “Assistant”) rather than drifting into story continuation.


### 4.1 - Qwen1.5-8B vs. GPT2

In the code below we’ll feed the same prompt to:

* **GPT-2 (completion-only)** – it will simply keep writing in the same style.  
* **Qwen-Chat (instruction-tuned)** – it will obey the instruction and respond directly.

Comparing the two outputs makes the difference easy to see.

In [62]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}
tokenizers, models = {}, {}
# Load models and tokenizers
for key, name in MODELS.items():
    tok = AutoTokenizer.from_pretrained(name)
    mdl = AutoModelForCausalLM.from_pretrained(name, torch_dtype=torch.float16 if torch.cuda.is_available() else None)
    if tok.pad_token_id is None:
        tok.pad_token = tok.eos_token
        mdl.config.pad_token_id = tok.eos_token_id
    mdl.eval().to("cuda" if torch.cuda.is_available() else "cpu")
    tokenizers[key] = tok
    models[key] = mdl


def _prepare_input_text(tok, model_key, prompt: str) -> str:
    # Use chat template for instruction-tuned Qwen; raw text for GPT-2
    if model_key == "qwen" and hasattr(tok, "apply_chat_template"):
        messages = [{"role": "user", "content": prompt}]
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

def generate(model_key, prompt, strategy="greedy", max_new_tokens=100,
             temperature=1.0, top_k=50, top_p=0.9, num_beams=5):
    tok, mdl = tokenizers[model_key], models[model_key]
    device = next(mdl.parameters()).device

    text = _prepare_input_text(tok, model_key, prompt)
    inputs = tok(text, return_tensors="pt").to(device)

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id,
    )

    if strategy == "greedy":
        gen_kwargs.update(do_sample=False)
    elif strategy == "top_k":
        gen_kwargs.update(do_sample=True, top_k=top_k, temperature=temperature)
    elif strategy == "top_p":
        gen_kwargs.update(do_sample=True, top_p=top_p, temperature=temperature)
    elif strategy == "beam":
        gen_kwargs.update(do_sample=False, num_beams=num_beams, early_stopping=True)
    else:
        raise ValueError("strategy must be one of: greedy | top_k | top_p | beam")

    with torch.no_grad():
        out_ids = mdl.generate(**inputs, **gen_kwargs)

    return tok.decode(out_ids[0], skip_special_tokens=True)

`torch_dtype` is deprecated! Use `dtype` instead!


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]


We downloaded two tiny checkpoints: `GPT‑2` (124 M parameters) and `Qwen‑1.5‑Chat` (1.8 B). If the cell took a while, that was mostly network time. Models are stored locally after the first run.

Let's now generate text and compare two models.


In [63]:

tests=[("Once upon a time","greedy"),("What is 2+2?","top_k"),("Suggest a party theme.","top_p")]
for prompt,strategy in tests:
    for key in ["gpt2","qwen"]:
        print(f"\n== {key.upper()} | {strategy} ==")
        print(generate(key,prompt,strategy,80))



== GPT2 | greedy ==


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== QWEN | greedy ==
system
You are a helpful assistant.
user
Once upon a time
assistant
Once upon a time, in a far-off land, there was a small village nestled among the rolling hills and lush green forests. The villagers were simple folk who lived off the land, farming crops and raising livestock for their sustenance. They had a deep respect for nature and its beauty, and they lived in harmony with the creatures that called this place home.

One day, a great drought struck the

== GPT2 | top_k ==
What is 2+2? Why does the word "half" mean "half" or "half a team"? 2+2 is a word for 2-1 between two opposing ends in a season. (Comp

# 5. (Optional) A Small LLM Playground

### 5.1 ‑ Interactive Playground

Enter a prompt, pick a model and decoding strategy, adjust the temperature, and press **Generate** to watch the model respond.


In [65]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Make sure models and tokenizers are loaded
try:
    tokenizers
    models
except NameError:
    raise RuntimeError("Please run the earlier setup cells that load the models before using the playground.")

# --- Helpers ---
def _prepare_input_text(tok, model_key, prompt: str) -> str:
    # Use chat template for instruction-tuned models (e.g., Qwen-Chat)
    if model_key in ("qwen", "qwen-chat", "qwen1.5") and hasattr(tok, "apply_chat_template"):
        messages = [{"role": "user", "content": prompt}]
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

def _generate(model_key: str, prompt: str, strategy="greedy",
              temperature=1.0, top_k=50, top_p=0.9, num_beams=5, max_new_tokens=100) -> str:
    tok = tokenizers[model_key]
    mdl = models[model_key]
    device = next(mdl.parameters()).device

    text = _prepare_input_text(tok, model_key, prompt)
    inputs = tok(text, return_tensors="pt").to(device)

    gen_kwargs = dict(
        max_new_tokens=int(max_new_tokens),
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id,
    )

    if strategy == "greedy":
        gen_kwargs.update(do_sample=False)
    elif strategy == "top_k":
        gen_kwargs.update(do_sample=True, top_k=int(top_k), temperature=float(temperature))
    elif strategy == "top_p":
        gen_kwargs.update(do_sample=True, top_p=float(top_p), temperature=float(temperature))
    elif strategy == "beam":
        gen_kwargs.update(do_sample=False, num_beams=int(num_beams), early_stopping=True)
    else:
        raise ValueError("strategy must be one of: greedy | top_k | top_p | beam")

    with torch.no_grad():
        out_ids = mdl.generate(**inputs, **gen_kwargs)

    return tok.decode(out_ids[0], skip_special_tokens=True)

# Your code to build boxes, dropdowns, and other elements in the UI using widgets and creating the UI using widgets.vbox and display.
# Refer to https://ipywidgets.readthedocs.io/en/stable/
# --- UI ---
model_dd = widgets.Dropdown(
    options=[(k.upper(), k) for k in tokenizers.keys()],
    value=next(iter(tokenizers.keys())),
    description="Model:",
)

strategy_dd = widgets.Dropdown(
    options=[("greedy", "greedy"), ("top_k", "top_k"), ("top_p", "top_p"), ("beam", "beam")],
    value="greedy",
    description="Strategy:",
)

temp_slider = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description="Temp")
topk_slider = widgets.IntSlider(value=50, min=1, max=200, step=1, description="top_k")
topp_slider = widgets.FloatSlider(value=0.9, min=0.05, max=1.0, step=0.01, readout_format=".2f", description="top_p")
beams_slider = widgets.IntSlider(value=5, min=2, max=10, step=1, description="beams")
maxnew_slider = widgets.IntSlider(value=80, min=1, max=512, step=1, description="Max new")

prompt_box = widgets.Textarea(
    value="Ask me anything…",
    placeholder="Type your prompt here",
    description="Prompt:",
    layout=widgets.Layout(width="100%", height="120px"),
)

gen_button = widgets.Button(description="Generate", button_style="primary")
out = widgets.Output(layout={"border": "1px solid #ccc"})

# Dynamic control visibility
def _toggle_controls(*_):
    st = strategy_dd.value
    topk_slider.layout.display = "flex" if st == "top_k" else "none"
    topp_slider.layout.display = "flex" if st == "top_p" else "none"
    beams_slider.layout.display = "flex" if st == "beam" else "none"
    temp_slider.layout.display  = "flex" if st in ("top_k", "top_p") else "none"

strategy_dd.observe(_toggle_controls, names="value")
_toggle_controls()

def _on_click(_):
    out.clear_output()
    with out:
        try:
            text = _generate(
                model_key=model_dd.value,
                prompt=prompt_box.value,
                strategy=strategy_dd.value,
                temperature=temp_slider.value,
                top_k=topk_slider.value,
                top_p=topp_slider.value,
                num_beams=beams_slider.value,
                max_new_tokens=maxnew_slider.value,
            )
            display(Markdown(f"**Output:**\n\n{text}"))
        except Exception as e:
            display(Markdown(f"**Error:** `{type(e).__name__}: {e}`"))

gen_button.on_click(_on_click)

controls_row1 = widgets.HBox([model_dd, strategy_dd, maxnew_slider])
controls_row2 = widgets.HBox([temp_slider, topk_slider, topp_slider, beams_slider])

ui = widgets.VBox([prompt_box, controls_row1, controls_row2, gen_button, out])
display(ui)



## 🎉 Congratulations!

You’ve just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen1.5-8B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
